In [ ]:
!nvidia-smi

Sun Nov  1 11:02:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |    111MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from __future__ import division
from numba import cuda, float64, jit
import time
import numpy as np
import math

thread = 32
size = 100


In [ ]:
g_time = 0.
c_time = 0.
n = 10 # количество иттераций
size = 100
#генерируем матрицы
for _ in range(n):
  A = np.random.randint(5, size=(size, size))
  A = A.astype(np.float64)
  B = np.random.randint(10, size=(size, size))
  B = B.astype(np.float64)

  start = time.time()
#передаем на gpu матрицы
  A_global_mem = cuda.to_device(A)
  B_global_mem = cuda.to_device(B)
  C_global_mem = cuda.device_array((size, size))

#задаем параметры куды, количество нитей в блоке, количество блоков в гриде  
  threadsperblock = (TPB, TPB)
  blockspergrid_x = math.ceil(A.shape[0] / threadsperblock[1])
  blockspergrid_y = math.ceil(B.shape[1] / threadsperblock[0])
  blockspergrid = (blockspergrid_x, blockspergrid_y)

#запускает gpu device
  mult_gpu_device[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
 #копируем матрицу на хост, чтобы показать 
  C_g = C_global_mem.copy_to_host()

  g_time += time.time() - start
#последовательное выполнение на процессоре
  start = time.time()
  C_c = mult_cpu_np(A,B)

  c_time += time.time() - start
#среднее время работы gpu, cpu и размер
print('GPU:',g_time/n)
print('CPU:',c_time/n)
print('size', size)



GPU: 0.001988387107849121
CPU: 0.0006983041763305664
size 100
